# CLEAR

Chargemaster Location-based Exploration for Affordability & Reform

This is the notebook file primarily responsible for pre-processing data, attaching important information, and generating database files for the github page. Below you can find all information about how data is processed from the downloaded `.csv` files found on most hospital sites. This is an exploratory project focused on creating interactive visualzations and tools to better inform people about their healthcare. The repo can always be maintained by downloading the most current year data for the specific hospital and putting it through the scripts. It should be noted that this is NOT a comprehensive list, but it can potentially be scaled to a full working-standalone site with enough time. 

All pre-processing code is written in python. See the `.html` files for how the D3 visualizations work. 

## How it works (Copied from README)

Hospitals that have been added to this 'web-app' are stored in a `.csv` file for quick look up and ease of access. This points to the loc of it's Charge Master `.json` file which is then queried for the specific procedure. Hospitals are gathered from the CSV list based on a radius look-up provided by the user. If a hospital in the radius does not offer the service, it will not display the price point compared to others in the radius. 

Currently limited to 500 procedures due to file size limits and me not wanted to set up a database for this. Parquet only works server side so i can't do iterative testing before publishing to pages, and pages deployments can take a while.  

## List of Hospitals

These are the hospital's which data has been gathered and processed for thus far:

| State    | Hospital Name                     | Zipcode     | Date                 | File Size    | Link                                                            |
|----------|--------------------------------|-------------|-------------------|-------------|------------------------------------------------|
| NC        | Duke University Hospital     |     27710    |      09/2025      |   3.32 GB   |                                                                   |
| NC        | Wake Med                           |                   |                          |                   |                                                                 |
| NC        | REX UNC                             |                   |                          |                   |                                                                 |

## Outside Sources Used

- zip_centroids.csv courtesy of SimpleMaps data https://simplemaps.com/data/us-zips.
- CMS.gov data for top 200 HCPCS and CPT codes billed for 2024 & top 100 lab codes. [Link](https://www.cms.gov/data-research/statistics-trends-and-reports/medicare-fee-for-service-parts-a-b/medicare-utilization-part-b)



***

## Data Processing

CSV files are too large to store on github, thus they are downloaded locally, converted to the necessary format, then uploaded. If you want to perform conversions yourself you will need to find the specific hospital chargemaster and document in the notebook accordingly.

Not all Charge Masters (CM) are formatted the same, as such, to keep this notebook from growing too large, custom python scripts will be made for unique CM's. This matters beccause some hospitals are regional or statewide 'chains' but can vary prices between locations. For example, 

**AdventHealth**
- AdventHealth Orlando
- AdventHealth Tampa
- AdventHealth Hendersonville

all are AdventHealth hospitals, but their prices and available procedures vary per location. However, the same script to clean and process their CM's works because the file structure doesn't change from loc to loc. Normally CM structure only changes from hospital to hospital (brand-wise), but I haven't looked at the majority of US hospitals so this statement might need to be amended. 

Think of this file as more of a "**Controller**" for the cleaning, while the cleaning process is performed by imported functions. Subsections from here on are labeled by State, be sure to check which Hospitals are in each subsection before uploading data. 



In [27]:
# hospitals.csv updater/editor
import hashlib
import requests
import json
from geopy.geocoders import Nominatim
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

geolocator = Nominatim(user_agent="CLEAR-geoapi-2025")
csv_file = 'docs/data/hospitals.csv'
df = pd.read_csv(csv_file)

# construct address for geocoding only (don't modify original data)
def construct_geocoding_address(row):
    # Build clean address from original components
    address = f"{row['address']}, {row['city']}, {row['state']} {row['zip']}"
    return address

# get lat/lon from address with increased timeout and retry/delay
def get_lat_lon(address, max_retries=3, delay=2):
    for attempt in range(max_retries):
        try:
            location = geolocator.geocode(address, timeout=5)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocoding {address} (attempt {attempt+1}): {e}")
            time.sleep(delay)
    return None, None

# generate short unique ID based on ['hospital'] + full composite address (base36, 8 chars)
def generate_short_id(row):
    full_address = construct_geocoding_address(row)
    unique_string = f"{row['name']}_{full_address}"
    hash_int = int(hashlib.md5(unique_string.encode()).hexdigest(), 16)
    short_id = base36encode(hash_int)[:8]
    return short_id

# base36 encoding for shorter IDs
def base36encode(number):
    chars = '0123456789abcdefghijklmnopqrstuvwxyz'
    if number == 0:
        return '0'
    result = ''
    while number > 0:
        number, i = divmod(number, 36)
        result = chars[i] + result
    return result

# Add lat/lon and short_id to dataframe, set json_path to be '/data/prices/['state']/['id'].json'
def update_dataframe(df):
    
    # Don't modify the address column - just use it for geocoding
    def lat_lon_with_delay(row):
        geocoding_address = construct_geocoding_address(row)
        lat, lon = get_lat_lon(geocoding_address)
        time.sleep(1)  # 1 second delay per request
        return pd.Series([lat, lon])
    
    df[['lat', 'lon']] = df.apply(lat_lon_with_delay, axis=1)
    df['id'] = df.apply(generate_short_id, axis=1)
    df['json_path'] = df.apply(lambda row: f"docs/data/prices/{row['state']}/{row['id']}.json", axis=1)
    df.to_csv(csv_file, index=False)
    
    return

update_dataframe(df)

In [ ]:
# now we need to create comparison df's for the top 200 HCPCS and CMS codes billed for 2024 & top 100 lab codes
# first load the codes from the .csv files
hcpcs_codes = pd.read_csv('docs/data/hcpcs_lvl2_top_200_codes_2024.csv')
lab_codes = pd.read_csv('docs/data/lab_top_100_codes_2024.csv')
cpt_codes = pd.read_csv('docs/data/cpt_lvl1_top_200_codes_2024.csv')

,Rank by Charges,HCPCS Code,Allowed Charges,Allowed Services,Unnamed: 4
0,1,99214,"12,493,376,407","103,756,876",NaN
1,2,99213,"5,914,372,895","69,301,624",NaN
2,3,99233,"2,693,744,916","22,975,112",NaN
3,4,99232,"2,676,454,801","34,687,153",NaN
4,5,99215,"2,166,116,667","12,926,784",NaN


In [28]:
# RUN TO LOAD HOSPITALS CSV
import os

# csv's are stored locally outside of CLEAR repo
# set up one folder then into 'ChargeMaster_Project/csv_files/'
# get path to csv_files folder outside CLEAR repo
workspace_root = os.path.dirname(os.path.abspath('CLEAR.ipynb'))
csv_folder = os.path.join(workspace_root, '..', 'ChargeMaster_Project', 'csv_files')
csv_folder = os.path.abspath(csv_folder)

# define path to hospitals.csv
hospitals_csv = os.path.join(workspace_root, 'docs', 'data', 'hospitals.csv')
hospitals_csv = os.path.abspath(hospitals_csv)

# read hospitals.csv to get list of hospitals and their file paths
hospitals_df = pd.read_csv(hospitals_csv)

***
## North Carolina Hospitals

In [34]:

# ======================================================================
# --------------- DUKE HOSPITAL TESTING ----------------
# ======================================================================

# Grab row for Duke Hospital in Durham, NC
hos_name = 'Duke University Hospital'
matching_hospitals = hospitals_df[hospitals_df['name'] == hos_name]
if not matching_hospitals.empty:
    duke_row = matching_hospitals.iloc[0]
else:
    print(f"Hospital '{hos_name}' not found in the dataset")
    duke_row = None

# grab json path for Duke Hospital
duke_json_path = duke_row['json_path']

# load a single csv file from csv_folder for testing
test_csv_path = os.path.join(csv_folder, 'DukeHospital_Durham.csv')
duke_df = pd.read_csv(test_csv_path)

# remove duke_df Hospital, City, State, Address columns before converting to parquet
duke_df = duke_df.drop(columns=['Hospital', 'City', 'State', 'Address'])


#code_cols = ['code_1', 'code_2', 'code_3', 'code_4']
# Check matches for each code column against hcpcs_codes, cpt_codes, and lab_codes, iteratively
# for col in code_cols:
#     print(f"Checking matches for column: {col}")
#     hcpcs_matches = duke_df[duke_df[col].isin(hcpcs_codes['HCPCS Code'])]
#     cpt_matches = duke_df[duke_df[col].isin(cpt_codes['HCPCS Code'])]
#     lab_matches = duke_df[duke_df[col].isin(lab_codes['HCPCS Code'])]
#     print(f"  HCPCS matches: {len(hcpcs_matches)}")
#     print(f"  CPT matches: {len(cpt_matches)}")
#     print(f"  Lab matches: {len(lab_matches)}")

"""

    This actually shows that code_2 contains HCPCS codes and code_3 contains CPT codes
    Checking matches for column: code_1
        HCPCS matches: 0
        CPT matches: 0
        Lab matches: 0
    Checking matches for column: code_2
        HCPCS matches: 76966
        CPT matches: 0
        Lab matches: 19987
    Checking matches for column: code_3
        HCPCS matches: 0
        CPT matches: 772
        Lab matches: 0
    Checking matches for column: code_4
        HCPCS matches: 0
        CPT matches: 0
        Lab matches: 0

"""

# Duke Hospital CM Structure
# code_2/code_3 [columns 3, 5 --> 4, 6 contain type] contain HCPCS and CPT codes, so we use those for comparison against the top 200 lists
# Columns 13-24 contain payer, plan, and pricing info, so we want all of those as well as column 0 which is the 
# description of the code [used for regex matching on the front end]
# final columns to keep: 0, 3-6, 13-24
duke_df = duke_df.iloc[:, [0] + list(range(3, 7)) + list(range(13, 25))]

# actually lets go ahead and drop some columns to conserve space
duke_df = duke_df.drop(columns=['standard_charge_algorithm', 'additional_generic_notes'])

# now we can search duke_df['code_2'] and duke_df['code_2_type'] against hcpcs_codes , cpt_codes, and lab_codes
# first search hcpcs_codes
hcpcs_matches = duke_df[duke_df['code_2'].isin(hcpcs_codes['HCPCS Code'])]
cpt_matches = duke_df[duke_df['code_3'].isin(cpt_codes['HCPCS Code'])]
lab_matches = duke_df[duke_df['code_2'].isin(lab_codes['HCPCS Code'])]

# Combine all matches into one dataframe, drop duplicates
match_dfs = [df for df in [hcpcs_matches, cpt_matches, lab_matches] if not df.empty]

if match_dfs:
    all_matches = pd.concat(match_dfs, ignore_index=True).drop_duplicates()
else:
    # Create empty DataFrame with same structure as duke_df if no matches
    all_matches = pd.DataFrame(columns=duke_df.columns)

# There are some duplicate issues, mainly rows where no est. price are given, so lets remove enteries that don't have est. prices
all_matches = all_matches[all_matches['estimated_amount'].notna() & (all_matches['estimated_amount'] != '')]

# Save output data to json file for Duke json path
all_matches.to_json(duke_json_path, orient='records', lines=True)

# drop file/df from memory to save space
del duke_df
del duke_row
del test_csv_path

# ======================================================================


C:\Users\jcing\AppData\Local\Temp\ipykernel_19156\362494677.py:19: DtypeWarning: Columns (7,8,9,12,19) have mixed types. Specify dtype option on import or set low_memory=False.
  duke_df = pd.read_csv(test_csv_path)


In [33]:
hos_name = 'Duke University Hospital'
matching_hospitals = hospitals_df[hospitals_df['name'] == hos_name]
if not matching_hospitals.empty:
    duke_row = matching_hospitals.iloc[0]
else:
    print(f"Hospital '{hos_name}' not found in the dataset")
    duke_row = None

# grab json path for Duke Hospital
duke_json_path = duke_row['json_path']

# load a single csv file from csv_folder for testing
test_csv_path = os.path.join(csv_folder, 'DukeHospital_Durham.csv')
duke_df = pd.read_csv(test_csv_path)

# remove duke_df Hospital, City, State, Address columns before converting to parquet
duke_df = duke_df.drop(columns=['Hospital', 'City', 'State', 'Address'])

duke_df = duke_df.iloc[:, [0] + list(range(3, 7)) + list(range(13, 25))]
duke_df

C:\Users\jcing\AppData\Local\Temp\ipykernel_19156\55375193.py:14: DtypeWarning: Columns (7,8,9,12,19) have mixed types. Specify dtype option on import or set low_memory=False.
  duke_df = pd.read_csv(test_csv_path)


,description,code_2,code_2_type,code_3,code_3_type,standard_charge_gross,standard_charge_discounted_cash,payer_name,plan_name,standard_charge_dollar,standard_charge_percentage,standard_charge_algorithm,estimated_amount,methodology,additional_generic_notes,standard_charge_min,standard_charge_max
0,"HOOK, THROAT CLOSED TI .25IN 8.0MM",27800169,LOCAL,278.0,RC,2919.78,788.340,CIGNA [1107150],CIGNA INTERNATIONAL [110715007],NaN,69.2,NaN,2020.48776,percent of total billed charges,NaN,934.3296,2481.813
1,NUSINERSEN (PF) 12 MG/5 ML INTRATHECAL SOLUTION,J2326,HCPCS,250.0,RC,641811.20,173289.024,UHC,UHC HMO,539121.408,NaN,NaN,NaN,other,other,1231.9020,545539.520
2,"SPACER, CON/REV FEM POST NKII SZ4 4MM",27800169,LOCAL,278.0,RC,5790.00,1563.300,CIGNA [1107150],CIGNA GWH PPO [110715018],NaN,69.2,NaN,4006.68000,percent of total billed charges,NaN,1852.8000,4921.500
3,DIFLUPREDNATE 0.05 % EYE DROPS,J3490,HCPCS,250.0,RC,737.92,199.240,AETNA [1107164],AETNA OPEN ACCESS HMO [110716402],NaN,57.8,NaN,426.52000,percent of total billed charges,NaN,236.1344,627.232
4,"HOOK, THROAT CLOSED TI .25IN 8.0MM",27800169,LOCAL,278.0,RC,2919.78,788.340,CIGNA [1107150],CIGNA OPEN ACCESS HMO [110715008],NaN,69.2,NaN,2020.48776,percent of total billed charges,NaN,934.3296,2481.813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12543718,"SCREW, CAPRI LOCKING ANGLE",C1713,HCPCS,278.0,RC,324.00,87.480,UHC [1107151],UNITED HEALTHCARE CHOICE [110715102],NaN,60.7,NaN,196.66800,percent of total billed charges,NaN,103.6800,275.400
12543719,"SCREW, CAPRI LOCKING ANGLE",C1713,HCPCS,278.0,RC,324.00,87.480,UHC [1107151],UNITED HEALTHCARE EMPIRE PLAN [110715107],NaN,60.7,NaN,196.66800,percent of total billed charges,NaN,103.6800,275.400
12543720,"SCREW, CAPRI LOCKING ANGLE",C1713,HCPCS,278.0,RC,324.00,87.480,UHC [1107151],UNITED HEALTHCARE POS EPO [110715110],NaN,60.7,NaN,196.66800,percent of total billed charges,NaN,103.6800,275.400
12543721,"SCREW, CAPRI LOCKING ANGLE",C1713,HCPCS,278.0,RC,324.00,87.480,UHC [1107151],UNITED HEALTHCARE STUDENT [110715111],NaN,60.7,NaN,196.66800,percent of total billed charges,NaN,103.6800,275.400
